In [1]:
# import re
# import psycopg2
# import pinecone
# from sentence_transformers import SentenceTransformer
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate
# import numpy as np

# # Constants
# PINECONE_API_KEY = "9fbe58e4-9e72-4023-90eb-ba8d022916b5"  # Replace with your Pinecone API key
# INDEX_NAME = "smart-desk"  # Replace with your Pinecone index name
# NAMESPACE = "projects"  # Replace with your namespace
# MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
# OPENAI_API_KEY = "sk-proj-UnzdWuWBs7ZQRbRPiRCoT3BlbkFJhPM1p7DdZUMklcpnWK1S"  # Replace with your OpenAI API key

# # PostgreSQL database connection details
# DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
# DATABASE_USERNAME = "postgres"
# DATABASE_PASSWORD = "valign#123"
# DATABASE_DB = "python_test_poc"
# PORT = 5432

# # Initialize Pinecone client
# def initialize_pinecone():
#     pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
    
#     # Check if the index exists, otherwise create it
#     if INDEX_NAME not in pc.list_indexes().names():
#         pc.create_index(
#             name=INDEX_NAME,
#             dimension=768,  # Assuming you're using sentence-transformers (768 dimensions)
#             metric='cosine',
#             spec=pinecone.ServerlessSpec(
#                 cloud='aws',
#                 region='us-east-1'  # Specify your region
#             )
#         )
    
#     return pc.Index(INDEX_NAME)

# # Load Hugging Face model for embeddings
# def load_huggingface_model():
#     return SentenceTransformer(MODEL_NAME)

# # Connect to PostgreSQL and fetch schema information
# def fetch_database_schema():
#     conn = psycopg2.connect(
#         host=DATABASE_HOST,
#         user=DATABASE_USERNAME,
#         password=DATABASE_PASSWORD,
#         dbname=DATABASE_DB,
#         port=PORT
#     )
#     cur = conn.cursor()
    
#     # Fetch schema information
#     cur.execute("""
#         SELECT table_name, column_name, data_type 
#         FROM information_schema.columns 
#         WHERE table_schema = 'public'
#     """)
    
#     schema = {}
#     for table, column, dtype in cur.fetchall():
#         if table not in schema:
#             schema[table] = []
#         schema[table].append({"column": column, "data_type": dtype})
    
#     cur.close()
#     conn.close()
#     return schema

# # Extract relevant entities based on regex and column names
# def extract_entities(user_query, schema):
#     entities = {
#         'project_name': None,
#         'owner': None
#     }
    
#     # Basic patterns for entity extraction
#     project_pattern = re.compile(r'project\s+([a-zA-Z0-9_ ]+)', re.IGNORECASE)
#     owner_pattern = re.compile(r'owner\s+of\s+project\s+([a-zA-Z0-9_ ]+)', re.IGNORECASE)
    
#     # Extract project name
#     project_match = project_pattern.search(user_query)
#     if project_match:
#         entities['project_name'] = project_match.group(1).strip()
    
#     # Extract owner-related information
#     owner_match = owner_pattern.search(user_query)
#     if owner_match:
#         entities['owner'] = owner_match.group(1).strip()
    
#     return entities

# # Query Pinecone for relevant context and augment the input
# def query_pinecone_and_augment_input(user_input, entities, namespace):
#     embedding_model = load_huggingface_model()
#     pinecone_index = initialize_pinecone()
    
#     augmented_input = user_input
#     pinecone_data = {}

#     for entity_name, entity_value in entities.items():
#         if entity_value:
#             # Create embeddings and ensure they are in correct format
#             query_embedding = embedding_model.encode([entity_value])[0]
#             query_embedding = np.array(query_embedding, dtype=np.float32)  # Ensure correct type
            
#             print(f"Query Embedding for {entity_value}: {query_embedding}")

#             try:
#                 result = pinecone_index.query(
#                     namespace=namespace,
#                     vector=query_embedding.tolist(),  # Convert numpy array to list
#                     top_k=3,
#                     include_values=True,
#                     includeMetadata = True
#                 )
#                 print(f"Result from Pinecone for {entity_value}: {result}")
                
#                 matches = result.get('matches', [])
#                 if matches:
#                     # Extract unique values from matches
#                     unique_values = [match['metadata'].get('unique_value') for match in matches if 'metadata' in match]
#                     if unique_values:
#                         pinecone_data[entity_name] = unique_values
                        
#                         # Handle multiple matches by interacting with the user
#                         if len(unique_values) > 1:
#                             print(f"Multiple matches found for '{entity_value}':")
#                             for idx, unique_value in enumerate(unique_values):
#                                 print(f"{idx + 1}: {unique_value}")
                            
#                             while True:
#                                 selection = input(f"Please select the most relevant option for '{entity_value}' (1-{len(unique_values)}): ")
#                                 try:
#                                     selected_value = unique_values[int(selection) - 1]
#                                     augmented_input = augmented_input.replace(entity_value, selected_value)
#                                     break
#                                 except (IndexError, ValueError):
#                                     print("Invalid selection. Please choose a valid option.")
#                         else:
#                             augmented_input = augmented_input.replace(entity_value, unique_values[0])
#                 else:
#                     print(f"No matches found for {entity_value} in Pinecone.")
#             except Exception as e:
#                 print(f"Error querying Pinecone: {str(e)}")
#                 return f"Error querying Pinecone: {str(e)}", {}
    
#     return augmented_input, pinecone_data

# # Generate SQL query based on augmented input and schema
# def generate_sql_query(entities, schema):
#     if entities['project_name']:
#         project_name = entities['project_name']
#         sql_query = f"""
#             SELECT owner 
#             FROM projects 
#             WHERE project_name = '{project_name}'
#         """
#         return sql_query
#     return "Unable to generate SQL query based on the input."

# # Initialize OpenAI Chat model
# openai_model = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     model_name="gpt-3.5-turbo",
#     temperature=0.7,
#     max_tokens=150
# )

# # Create a ChatPromptTemplate with the knowledge base included
# template = """
# ## Knowledge Base:
# {knowledge_base}

# ## Database Schema:
# {database_schema}

# ## Question:
# {question}

# ## Answer:
# """

# prompt_template = ChatPromptTemplate.from_template(template)

# def get_answer_from_chatbot(question, database_schema):
#     try:
#         # Format the prompt with the knowledge base, schema, and user question
#         prompt = prompt_template.format(
#             knowledge_base="",
#             database_schema=database_schema,
#             question=question
#         )

#         # Generate response using OpenAI model
#         response = openai_model.invoke(input=prompt)

#         # Extract the response text
#         parsed_response = response.content.strip() if hasattr(response, 'content') else "No response content found."
#         return parsed_response

#     except Exception as e:
#         return f"Error generating response from OpenAI: {str(e)}"

# # Determine if user query is related to database or general knowledge
# def determine_query_type(user_query):
#     # Simple keyword-based logic to differentiate between database and general queries
#     if "owner" in user_query.lower() or "project" in user_query.lower():
#         return "database"
#     return "general"

# # Main function to handle user queries
# def process_user_query(user_query):
#     # Determine if the query is database-related
#     query_type = determine_query_type(user_query)
    
#     if query_type == "database":
#         # Fetch database schema
#         database_schema = fetch_database_schema()

#         # Extract entities from the user input
#         entities = extract_entities(user_query, database_schema)
#         print(f"Extracted Entities: {entities}")
        
#         if not any(entities.values()):
#             print("No relevant entities extracted from the query.")
#             return
        
#         # Query Pinecone and augment user input
#         augmented_input, pinecone_data = query_pinecone_and_augment_input(user_query, entities, NAMESPACE)
#         print(f"Queried Data from Pinecone: {pinecone_data}")
#         print(f"Augmented Input: {augmented_input}")

#         # Confirm the augmented input with the user before generating SQL query
#         confirmation = input(f"Is this augmented input correct? (yes/no): {augmented_input}: ")
#         if confirmation.lower() == "yes":
#             # Generate SQL query based on augmented input
#             sql_query = generate_sql_query(entities, database_schema)
#             print(f"Generated SQL Query: {sql_query}")
#         else:
#             print("Please rephrase your query and try again.")
#     else:
#         # Answer general questions using chatbot
#         database_schema = fetch_database_schema()
#         answer = get_answer_from_chatbot(user_query, database_schema)
#         print(f"Answer: {answer}")

# # Example usage
# if __name__ == "__main__":
#     while True:
#         question = input("Ask a question (type 'exit' to quit): ")
#         if question.lower() == "exit":
#             break
#         process_user_query(question)


In [ ]:
import re
import psycopg2
import pinecone
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import numpy as np

# Constants
PINECONE_API_KEY = ""  # Replace with your Pinecone API key
INDEX_NAME = "smart-desk"  # Replace with your Pinecone index name
NAMESPACE = "projects"  # Replace with your namespace
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
OPENAI_API_KEY = "sk-proj-UnzdWuWBs7ZQRbRPiRCoT3BlbkFJhPM1p7DdZUMklcpnWK1S"  # Replace with your OpenAI API key

# PostgreSQL database connection details
DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
DATABASE_USERNAME = "postgres"
DATABASE_PASSWORD = "valign#123"
DATABASE_DB = "python_test_poc"
PORT = 5432

# Initialize Pinecone client
def initialize_pinecone():
    pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
    
    # Check if the index exists, otherwise create it
    if INDEX_NAME not in pc.list_indexes().names():
        pc.create_index(
            name=INDEX_NAME,
            dimension=768,  # Assuming you're using sentence-transformers (768 dimensions)
            metric='cosine',
            spec=pinecone.ServerlessSpec(
                cloud='aws',
                region='us-east-1'  # Specify your region
            )
        )
    
    return pc.Index(INDEX_NAME)

# Load Hugging Face model for embeddings
def load_huggingface_model():
    return SentenceTransformer(MODEL_NAME)

# Connect to PostgreSQL and fetch schema information
def fetch_database_schema():
    conn = psycopg2.connect(
        host=DATABASE_HOST,
        user=DATABASE_USERNAME,
        password=DATABASE_PASSWORD,
        dbname=DATABASE_DB,
        port=PORT
    )
    cur = conn.cursor()
    
    # Fetch schema information
    cur.execute("""
        SELECT table_name, column_name, data_type 
        FROM information_schema.columns 
        WHERE table_schema = 'public'
    """)
    
    schema = {}
    for table, column, dtype in cur.fetchall():
        if table not in schema:
            schema[table] = []
        schema[table].append({"column": column, "data_type": dtype})
    
    cur.close()
    conn.close()
    return schema

# Extract relevant entities based on regex and column names
def extract_entities(user_query, schema):
    entities = {
        'project_name': None,
        'owner': None
    }
    
    # Basic patterns for entity extraction
    project_pattern = re.compile(r'project\s+([a-zA-Z0-9_ ]+)', re.IGNORECASE)
    owner_pattern = re.compile(r'owner\s+of\s+project\s+([a-zA-Z0-9_ ]+)', re.IGNORECASE)
    
    # Extract project name
    project_match = project_pattern.search(user_query)
    if project_match:
        entities['project_name'] = project_match.group(1).strip()
    
    # Extract owner-related information
    owner_match = owner_pattern.search(user_query)
    if owner_match:
        entities['owner'] = owner_match.group(1).strip()
    
    return entities

# Query Pinecone for relevant context and augment the input
def query_pinecone_and_augment_input(user_input, entities, namespace):
    embedding_model = load_huggingface_model()
    pinecone_index = initialize_pinecone()
    
    augmented_input = user_input
    pinecone_data = {}

    for entity_name, entity_value in entities.items():
        if entity_value:
            # Create embeddings and ensure they are in correct format
            query_embedding = embedding_model.encode([entity_value])[0]
            query_embedding = np.array(query_embedding, dtype=np.float32)  # Ensure correct type
            
            print(f"Query Embedding for {entity_value}: {query_embedding}")

            try:
                result = pinecone_index.query(
                    namespace=namespace,
                    vector=query_embedding.tolist(),  # Convert numpy array to list
                    top_k=3,
                    include_values=True,
                    includeMetadata = True
                )
                print(f"Result from Pinecone for {entity_value}: {result}")
                
                matches = result.get('matches', [])
                if matches:
                    # Extract unique values from matches
                    unique_values = [match['metadata'].get('unique_value') for match in matches if 'metadata' in match]
                    if unique_values:
                        pinecone_data[entity_name] = unique_values
                        
                        # Handle multiple matches by interacting with the user
                        if len(unique_values) > 1:
                            print(f"Multiple matches found for '{entity_value}':")
                            for idx, unique_value in enumerate(unique_values):
                                print(f"{idx + 1}: {unique_value}")
                            
                            while True:
                                selection = input(f"Please select the most relevant option for '{entity_value}' (1-{len(unique_values)}): ")
                                try:
                                    selected_value = unique_values[int(selection) - 1]
                                    augmented_input = augmented_input.replace(entity_value, selected_value)
                                    break
                                except (IndexError, ValueError):
                                    print("Invalid selection. Please choose a valid option.")
                        else:
                            augmented_input = augmented_input.replace(entity_value, unique_values[0])
                else:
                    print(f"No matches found for {entity_value} in Pinecone.")
            except Exception as e:
                print(f"Error querying Pinecone: {str(e)}")
                return f"Error querying Pinecone: {str(e)}", {}
    
    return augmented_input, pinecone_data

# Generate SQL query based on augmented input and schema
def generate_sql_query(entities, schema):
    if entities['project_name']:
        project_name = entities['project_name']
        sql_query = f"""
            SELECT owner 
            FROM projects 
            WHERE project_name = '{project_name}'
        """
        return sql_query
    return "Unable to generate SQL query based on the input."

# Initialize OpenAI Chat model
openai_model = ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=150
)

# Create a ChatPromptTemplate with the knowledge base included
template = """
## Knowledge Base:
{knowledge_base}

## Database Schema:
{database_schema}

## Question:
{question}

## Answer:
"""

prompt_template = ChatPromptTemplate.from_template(template)

def get_answer_from_chatbot(question, database_schema):
    try:
        # Format the prompt with the knowledge base, schema, and user question
        prompt = prompt_template.format(
            knowledge_base="",
            database_schema=database_schema,
            question=question
        )

        # Generate response using OpenAI model
        response = openai_model.invoke(input=prompt)

        # Extract the response text
        parsed_response = response.content.strip() if hasattr(response, 'content') else "No response content found."
        return parsed_response

    except Exception as e:
        return f"Error generating response from OpenAI: {str(e)}"

# Determine if user query is related to database or general knowledge
def determine_query_type(user_query):
    # Simple keyword-based logic to differentiate between database and general queries
    if "owner" in user_query.lower() or "project" in user_query.lower():
        return "database"
    return "general"

# Main function to handle user queries
def process_user_query(user_query):
    # Determine if the query is database-related
    query_type = determine_query_type(user_query)
    
    if query_type == "database":
        # Fetch database schema
        database_schema = fetch_database_schema()

        # Extract entities from the user input
        entities = extract_entities(user_query, database_schema)
        print(f"Extracted Entities: {entities}")
        
        if not any(entities.values()):
            print("No relevant entities extracted from the query.")
            return
        
        # Query Pinecone and augment user input
        augmented_input, pinecone_data = query_pinecone_and_augment_input(user_query, entities, NAMESPACE)
        print(f"Queried Data from Pinecone: {pinecone_data}")
        print(f"Augmented Input: {augmented_input}")

        # Confirm the augmented input with the user before generating SQL query
        confirmation = input(f"Is this augmented input correct? (yes/no): {augmented_input}: ")
        if confirmation.lower() == "yes":
            # Generate SQL query based on augmented input
            sql_query = generate_sql_query(entities, database_schema)
            print(f"Generated SQL Query: {sql_query}")
        else:
            print("Please rephrase your query and try again.")
    else:
        # Answer general questions using chatbot
        database_schema = fetch_database_schema()
        answer = get_answer_from_chatbot(user_query, database_schema)
        print(f"Answer: {answer}")

# Example usage
if __name__ == "__main__":
    while True:
        question = input("Ask a question (type 'exit' to quit): ")
        if question.lower() == "exit":
            break
        process_user_query(question)


Ask a question (type 'exit' to quit):  Hi


Answer: Hello! How can I assist you today?


Ask a question (type 'exit' to quit):  what is the status of a project IIFL Samasta


Extracted Entities: {'project_name': 'IIFL Samasta', 'owner': None}


C:\Users\Admin\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Query Embedding for IIFL Samasta: [ 2.13010348e-02 -2.55320892e-02 -5.87795721e-03 -1.36354018e-03
  3.42779644e-02  2.66339239e-02 -3.12309228e-02 -1.52818877e-02
 -8.64014775e-02 -1.15376208e-02  3.05121187e-02 -2.46994328e-02
  1.94623657e-02  4.38344479e-03  4.11561504e-03  2.50884220e-02
  6.08321326e-03 -2.59168390e-02 -2.60414891e-02 -3.93199101e-02
  1.97558161e-02 -5.44649921e-03  2.42393054e-02 -1.93107631e-02
  6.63040578e-03  6.19241921e-03  7.32292384e-02 -2.30317805e-02
  1.32525489e-02  1.46900676e-02  2.00841185e-02 -1.74006578e-02
 -1.43293915e-02 -2.90931277e-02  2.34568438e-06 -1.08414376e-02
  2.95128357e-02 -1.66468229e-02 -5.15728183e-02  5.83672225e-02
 -1.04434472e-02  1.40052084e-02 -5.86278737e-02 -2.53609270e-02
 -5.94092570e-02 -3.78906121e-03  2.63560098e-02  4.18794155e-02
  8.30237847e-03 -2.06549261e-02 -2.57454813e-02 -2.94394065e-02
 -5.41704260e-02 -2.03558840e-02 -6.34381995e-02  5.11399135e-02
 -2.88117696e-02 -4.21421826e-02 -1.87582728e-02  2.0042

Please select the most relevant option for 'IIFL Samasta' (1-3):  1


Queried Data from Pinecone: {'project_name': ['IIFl Samasta CPL CR', 'IIFL Samasta - CGRM', 'IIFL SAMASTA - RPA BOT']}
Augmented Input: what is the status of a project IIFl Samasta CPL CR


Is this augmented input correct? (yes/no): what is the status of a project IIFl Samasta CPL CR:  yes


Generated SQL Query: 
            SELECT owner 
            FROM projects 
            WHERE project_name = 'IIFL Samasta'
        


In [ ]:
# import psycopg2
# import pinecone
# from sentence_transformers import SentenceTransformer
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate

# # Constants
# PINECONE_API_KEY = "9fbe58e4-9e72-4023-90eb-ba8d022916b5"  # Replace with your Pinecone API key
# INDEX_NAME = "smart-desk"  # Replace with your Pinecone index name
# NAMESPACE = "projects"  # Replace with your namespace
# MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
# OPENAI_API_KEY = "sk-proj-UnzdWuWBs7ZQRbRPiRCoT3BlbkFJhPM1p7DdZUMklcpnWK1S"  # Replace with your OpenAI API key

# # PostgreSQL database connection details
# DATABASE_HOST = "database-test-postgress-instance.cpk2uyae6iza.ap-south-1.rds.amazonaws.com"
# DATABASE_USERNAME = "postgres"
# DATABASE_PASSWORD = "valign#123"
# DATABASE_DB = "python_test_poc"
# PORT = 5432

# # Initialize Pinecone client
# def initialize_pinecone():
#     pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)
#     return pc.Index(INDEX_NAME)

# # Load Hugging Face model for embeddings
# def load_huggingface_model():
#     return SentenceTransformer(MODEL_NAME)

# # Connect to PostgreSQL and fetch schema information
# def fetch_database_schema():
#     conn = psycopg2.connect(
#         host=DATABASE_HOST,
#         user=DATABASE_USERNAME,
#         password=DATABASE_PASSWORD,
#         dbname=DATABASE_DB,
#         port=PORT
#     )
#     cur = conn.cursor()
    
#     # Fetch schema information
#     cur.execute("""
#         SELECT table_name, column_name, data_type 
#         FROM information_schema.columns 
#         WHERE table_schema = 'public'
#     """)
    
#     schema = {}
#     for table, column, dtype in cur.fetchall():
#         if table not in schema:
#             schema[table] = []
#         schema[table].append({"column": column, "data_type": dtype})
    
#     cur.close()
#     conn.close()
#     return schema

# # Query Pinecone for relevant context
# def query_pinecone(query, namespace):
#     embedding_model = load_huggingface_model()
#     query_embedding = embedding_model.encode([query])[0]

#     # Initialize Pinecone index
#     index = initialize_pinecone()
    
#     try:
#         # Query Pinecone
#         result = index.query(
#             namespace=namespace,
#             vector=query_embedding,
#             top_k=3,
#             include_values=True
#         )
#     except Exception as e:
#         return f"Error querying Pinecone: {str(e)}"
    
#     # Extract and return relevant context from results
#     matches = result.get('matches', [])
    
#     if not matches:
#         return "No relevant context found."
    
#     # Extract values
#     context = ""
#     for item in matches:
#         values = item.get('values', [])
#         if isinstance(values, list):
#             context += "\n".join(map(str, values)) + "\n"
#         else:
#             context += str(values) + "\n"
    
#     return context.strip()

# # Generate SQL query based on user input and schema
# def generate_sql_query(user_query, schema):
#     if "owner" in user_query.lower() and "project" in user_query.lower():
#         project_name = user_query.split("project")[-1].strip().strip("?").strip()
#         sql_query = f"""
#             SELECT owner 
#             FROM projects 
#             WHERE project_name = '{project_name}'
#         """
#         return sql_query
    
#     return "Unable to generate SQL query based on the user input."

# # Initialize OpenAI Chat model
# openai_model = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     model_name="gpt-3.5-turbo",
#     temperature=0.7,
#     max_tokens=150
# )

# # Create a ChatPromptTemplate with the knowledge base included
# template = """
# ## Knowledge Base:
# {knowledge_base}

# ## Database Schema:
# {database_schema}

# ## Question:
# {question}

# ## Answer:
# """

# prompt_template = ChatPromptTemplate.from_template(template)

# def get_answer_from_chatbot(question):
#     try:
#         # Fetch database schema
#         database_schema = fetch_database_schema()
        
#         # Query Pinecone to get relevant context
#         knowledge_base = query_pinecone(question, NAMESPACE)
        
#         # Format the prompt with the knowledge base, schema, and user question
#         prompt = prompt_template.format(
#             knowledge_base=knowledge_base,
#             database_schema=database_schema,
#             question=question
#         )

#         # Generate response using OpenAI model
#         response = openai_model.invoke(input=prompt)

#         # Extract the response text
#         parsed_response = response.content.strip() if hasattr(response, 'content') else "No response content found."
#         return parsed_response

#     except Exception as e:
#         return f"Error generating response from OpenAI: {str(e)}"

# # Determine if user query is related to database or general knowledge
# def determine_query_type(user_query):
#     # Simple keyword-based logic to differentiate between database and general queries
#     if "owner" in user_query.lower() or "project" in user_query.lower():
#         return "database"
#     return "general"

# # Example usage
# if __name__ == "__main__":
#     while True:
#         question = input("Ask a question (type 'exit' to quit): ")
#         if question.lower() == 'exit':
#             break
        
#         query_type = determine_query_type(question)
        
#         if query_type == "database":
#             # Generate SQL query based on user input
#             sql_query = generate_sql_query(question, fetch_database_schema())
#             print(f"Generated SQL Query: {sql_query}")
#         else:
#             sql_query = "N/A"
        
#         # Get answer from chatbot
#         answer = get_answer_from_chatbot(question)
#         print(f"Answer: {answer}")


In [ ]:
# # #Sample ChatBot
# import pinecone
# from pinecone.grpc import PineconeGRPC as Pinecone
# from sentence_transformers import SentenceTransformer
# from langchain_openai import ChatOpenAI
# from langchain.prompts import ChatPromptTemplate

# # Constants
# PINECONE_API_KEY = "9fbe58e4-9e72-4023-90eb-ba8d022916b5"  # Replace with your Pinecone API key
# INDEX_NAME = "smart-desk"  # Replace with your Pinecone index name
# NAMESPACE = "projects"  # Replace with your namespace
# MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
# OPENAI_API_KEY = "sk-proj-UnzdWuWBs7ZQRbRPiRCoT3BlbkFJhPM1p7DdZUMklcpnWK1S"  # Replace with your OpenAI API key

# # Initialize Pinecone client
# def initialize_pinecone():
#     pc = Pinecone(api_key=PINECONE_API_KEY)
#     return pc.Index(INDEX_NAME)

# # Load Hugging Face model for embeddings
# def load_huggingface_model():
#     embedding_model = SentenceTransformer(MODEL_NAME)
#     return embedding_model

# # Query Pinecone for relevant context
# def query_pinecone(query, namespace):
#     embedding_model = load_huggingface_model()
#     query_embedding = embedding_model.encode([query])[0]

#     # Initialize Pinecone index
#     index = initialize_pinecone()
    
#     # Query Pinecone
#     result = index.query(
#         namespace=namespace,
#         vector=query_embedding,
#         top_k=3,
#         include_values=True
#     )
    
#     # Extract and return relevant context from results
#     matches = result.get('matches', [])
    
#     if not matches:
#         return "No relevant context found."
    
#     # Extract values
#     context = ""
#     for item in matches:
#         values = item.get('values', [])
#         if isinstance(values, list):
#             context += "\n".join(map(str, values)) + "\n"
#         else:
#             context += str(values) + "\n"
    
#     return context.strip()

# # Initialize OpenAI Chat model
# openai_model = ChatOpenAI(
#     openai_api_key=OPENAI_API_KEY,
#     model_name="gpt-3.5-turbo",
#     temperature=0.7,
#     max_tokens=150
# )

# # Create a ChatPromptTemplate with the knowledge base included
# template = """
# ## Knowledge Base:
# {knowledge_base}

# ## Question:
# {question}

# ## Answer:
# """

# prompt_template = ChatPromptTemplate.from_template(template)

# def get_answer_from_chatbot(question):
#     # Query Pinecone to get relevant context
#     knowledge_base = query_pinecone(question, NAMESPACE)
    
#     # Format the prompt with the knowledge base and the user question
#     prompt = prompt_template.format(
#         knowledge_base=knowledge_base,
#         question=question
#     )

#     # Generate response using OpenAI model
#     response = openai_model.invoke(input=prompt)

#     # Extract the response text
#     parsed_response = response.content.strip() if hasattr(response, 'content') else "No response content found."
#     return parsed_response

# # Example usage
# if __name__ == "__main__":
#     while True:
#         question = input("Ask a question (type 'exit' to quit): ")
#         if question.lower() == 'exit':
#             break
#         answer = get_answer_from_chatbot(question)
#         print(f"Answer: {answer}")
